In [1]:
import open3d as o3d
import numpy as np

data=np.load("3d_shape_points_data.npz")
points = data['points']

print(f'shape of the numpy array {points.shape}')
print(points)

# Create an Open3D point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Apply MLS smoothing
pcd_smoothed = pcd.voxel_down_sample(voxel_size=0.5)
o3d.io.write_point_cloud("smoothed.ply", pcd_smoothed)
# o3d.visualization.draw_geometries([pcd_smoothed])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
shape of the numpy array (65536, 3)
[[ 9.998086    0.          0.        ]
 [ 9.994803    0.24535865  0.        ]
 [ 9.985485    0.4905554   0.        ]
 ...
 [19.84671    -1.4639814  40.        ]
 [19.879618   -0.97662306 40.        ]
 [19.900478   -0.48852932 40.        ]]


True

In [2]:
# remove outliers if any

cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1)
filtered_pcd = pcd.select_by_index(ind)
# o3d.visualization.draw_geometries([filtered_pcd], window_name="Filtered Point Cloud")
pcd=filtered_pcd

In [3]:
# creating a mesh from the point cloud using ball pivoting method

radii = [0.1,0.2,0.3] 
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid( radius=0.1, max_nn=20)) # normals is required for ball pivoting method

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

# Compute vertex normals
mesh.compute_vertex_normals()

# o3d.visualization.draw_geometries([mesh])

TriangleMesh with 53598 points and 106458 triangles.

In [4]:
# some preprocessing on the mesh
mesh = mesh.filter_smooth_laplacian(number_of_iterations=100)

# Recompute normals for better visualization
mesh.compute_vertex_normals()

mesh = mesh.filter_smooth_simple(number_of_iterations=5)
mesh = mesh.filter_smooth_taubin(number_of_iterations=5)
# Visualize the smoothed mesh
# o3d.visualization.draw_geometries([mesh])

o3d.io.write_triangle_mesh("temp_mesh.ply", mesh)

True

In [5]:
# to fill the holes in the mesh

# import pymeshlab

# ms = pymeshlab.MeshSet()
# ms.load_new_mesh("temp_mesh.obj")
# ms.generate_alpha_wrap(alpha_fraction= 0.03, offset_fraction  = 0.001)
# ms.save_current_mesh("dest_mesh.stl")


In [6]:
import numpy as np
import pyvista as pv

from pymeshfix import PyTMesh

filename = "temp_mesh.ply"

orig_mesh = pv.read(filename)
mfix = PyTMesh(False)
mfix.load_file(filename)

# Fills all the holes having at least 'nbe' boundary edges. If
# 'refine' is true, adds inner vertices to reproduce the
# sampling density of the surroundings. Returns number of holes
# patched.  If 'nbe' is 0 (default), all the holes are patched.
mfix.fill_small_boundaries(nbe=100, refine=True)

vert, faces = mfix.return_arrays()

triangles = np.empty((faces.shape[0], 4))
triangles[:, -3:] = faces
triangles[:, 0] = 3

# meshnew = pv.PolyData(vert, triangles)
# meshnew.plot()

: 

In [ ]:
# Load the STL file
mesh = o3d.io.read_triangle_mesh("dest_mesh.stl")
# Visualize the mesh
o3d.visualization.draw_geometries([mesh])

In [ ]:
# creating the point cloud back from the meshes
point_cloud = mesh.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([point_cloud])

In [11]:
mesh.compute_vertex_normals()

# Poisson reconstruction (better for smooth surfaces)
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(mesh.sample_points_poisson_disk(5000), depth=8)

# Visualize
o3d.visualization.draw_geometries([mesh])